<div align="right"><i>Peter Norvig<br>2012; updated 18 August 2019</i></div>

# Weighing Twelve Balls on a Balance Scale

> *You are given twelve identical-looking balls and a two-sided scale. One of the balls is of a different weight, although you don't know whether it's lighter or heavier. How can you use just three weighings of the scale to determine not only what the different ball is, but also whether it's lighter or heavier?*

This is a traditional brain-teaser puzzle, meant to be solved with paper and pencil. 
But I want to solve not just this specific puzzle, but related puzzles where you can vary (a) the number of balls, (b) the number of weighings allowed, and (c) whether the odd ball might be heavier, lighter, or either, or neither. For that I'll ned a program. (I originally solved wrote this program in 2012, but am republishing it here in revised form because the problem was mentioned in the [538 Riddler](https://fivethirtyeight.com/features/which-billiard-ball-is-rigged/) for 16 August 2019.)

# Design

Here are the concepts I'm dealing with:

- **balls**: In the general case I have N balls. I'll represent with a list like `[1, 2, 3]` for N = 3.
- **oddballs**: Exactly one of the balls is **odd** in its weight. 
I'll represent the situation where ball N is heavier as +N, and where ball N is lighter as -N. (I'll represent the situation where no ball is odd with `0`; that's not needed for the puzzle stated above, but is a nice extension.) With N = 3, I will represent the set of all possible oddballs as `{+1, -1, +2, -2, +3, -3}`. 
- **puzzle**: I'll call `Puzzle(N, W, {-1, +1})` to create a puzzle with N balls, W weighings allowed, and the oddballs allowing for each ball to be lighter (-1) or heavier (+1). The third argument can be any subset of `{-1, 0, +1}`, where the `0` stands for "allow the possibility that no ball is odd; they all weigh the same." 
- **weighing**: I can weigh a collection of balls on the left versus a collection on the right, and the result will be that the left side is greater than, equal to, or less than the right in weight. I'll denote that with the call `weigh(L, R, oddball)`, which returns a string, `'gt'`, `'eq'`, or `'lt'`.
- **weight**: I'll arbitrarily say that a normal ball weighs 100, a lighter ball 99, and a heavier ball 101.
- **solution**: Given a puzzle, a solution is a **strategy tree** that can correctly discover the odd ball, whatever it is, in the allowable number of weighings.
- **strategy tree**: a tree where each node is either a leaf node which is an *oddball* integer, or is an interior node with 5 components:  `Tree(L, R, gt, eq, lt)` is a node where `L` and `R` are the collections of balls to be weighed, and `gt`, `eq`, and `lt` are trees for when the result of the weighing is that the total weight of the balls on the left is greater than, equal to, or less than, respectively, the total weight of the balls on the right.
- **following a path in a tree**: I'll use `follow(tree, oddball)` to say "follow the path through the tree, doing each weighing under the assumption of the given oddball, and return the leaf node reached by the path&mdash;the oddball that the tree predicts. Note that the function `follow` gets to see what the oddball is, but the `tree` never gets direct access to that; the tree has to figure it out by doing weighings.
- **valid tree**: a tree is a valid solution to a puzzle if no branch uses more than the allowable number of weighings, and if, for every possible oddball in the puzzle, following the path through the tree correct returns that oddball as the answer. I'll use `valid(tree, puzzle)` for this.


# Implementation

Let's start implementing the concepts:

In [1]:
from collections import namedtuple
import random

#### Types

Ball = Oddball = int

class Puzzle:
    "Represent a specific ball-weighing puzzle."
    def __init__(self, N=12, weighings=3, oddities={-1, +1}):
        self.weighings = weighings
        self.balls     = list(range(1, N + 1))
        self.oddballs  = {b * o for b in self.balls for o in oddities}
        
Tree = namedtuple('Tree', 'L, R, gt, eq, lt')

#### Functions
    
def weigh(L, R, oddball) -> str:
    "Weigh balls L against balls R, given the oddball; return 'gt', 'eq', or 'lt'."
    diff = sum(weight(b, oddball) for b in L) - sum(weight(b, oddball) for b in R)
    return ('gt' if diff > 0 else
            'lt' if diff < 0 else
            'eq')

def weight(ball, oddball) -> int: 
    return 101 if +ball == oddball else 99 if -ball == oddball else 100
    
def solve(puzzle) -> Tree or None:
    "Return a valid tree; one that solves the puzzle, or None."
    tree = find_tree(puzzle, puzzle.oddballs, puzzle.weighings)
    return tree if valid(tree, puzzle) else None
    
def follow(tree, oddball) -> Oddball:
    "Follow a path through the tree and return the oddball that the tree leads us to."
    if isinstance(tree, Oddball):
        return tree
    else:
        result = weigh(tree.L, tree.R, oddball)
        return follow(getattr(tree, result), oddball)
    
def valid(tree, puzzle) -> bool:
    "Does the strategy tree solve the puzzle correctly for all possible oddballs?"
    return (tree is not None and
            depth(tree) <= puzzle.weighings and 
            all(follow(tree, oddball) == oddball 
                for oddball in puzzle.oddballs))

def depth(tree) -> int:
    "Maximum depth of a strategy tree."
    return (0 if isinstance(tree, Oddball) else 
            1 + max(depth(tree.gt), depth(tree.eq), depth(tree.lt)))

Let's try out some of these functions:

In [2]:
p12 = Puzzle(12) # The original puzzle with 12 balls

print(p12.weighings)
print(p12.balls)
print(p12.oddballs)

3
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, -12, -11, -1, -9, -8, -7, -6, -5, -4, -3, -2, -10}


In [3]:
# If we weigh balls 1, 2 against 3, 4, and the oddball is that 5 is lighter, the result should be 'eq'
weigh([1, 2], [3, 4], -5)

'eq'

In [4]:
weight(1, oddball=-5)

100

In [5]:
weight(5, oddball=-5)

99

# Strategy for Finding a Valid Tree

Now for the tricky part. We want to find a valid tree to solve a puzzle. The key idea is that a **weighing** gives us information by making a **partition** of the possible **oddballs** into entries for eaach of the three possible weighing results: `gt`, `eq`, or `lt`. Subsequent subtrees can handle each of the partitions.


In [6]:
def partition(L, R, oddballs) -> dict:
    "Build a dict of the possible outcomes (oddballs) from weighing L versus R."
    part = dict(gt=set(), eq=set(), lt=set())
    for odd in oddballs:
        part[weigh(L, R, odd)].add(odd)
    return part

For example, with 12 balls, if we weigh balls 1 and 2 on the left versus 11 and 12 on the right, then there are 4 ways  the left side can be greater than the right: either 1 or 2 is heavier or 11 or 12 is lighter. Similarly there are 4 ways of getting a `lt` weighing result. The remaining 16 possible oddballs&mdash;balls 3 through 10 being either heavier or lighter&mdash;show up in the `eq` entry of the partition:

In [7]:
partition([1, 2], [11, 12], p12.oddballs)

{'gt': {-12, -11, 1, 2},
 'eq': {-10, -9, -8, -7, -6, -5, -4, -3, 3, 4, 5, 6, 7, 8, 9, 10},
 'lt': {-2, -1, 11, 12}}

If this was the first weighing in our strategy tree, could we go on to solve the puzzle in 3 weighings? **No!** Any one weighing can at best partition the remaining possibilities into 3 equal entries. To solve the puzzle, we need every path in the tree to end up with only one possibility. So any two weighings can handle at most  3 &times; 3 = 9 possibilities; here we have 16, which is too many. We call this a **bad partition**.

The following is a **good partition** because each of the entries has 8 possibilities, and 8 is less than 9. (Note: being a good partition does not guarantee that the problem is solvable from there; but being a bad partition guarantees that it is not solvable.)

In [8]:
partition([1, 2, 3, 4], [9, 10, 11, 12], p12.oddballs)

{'gt': {-12, -11, -10, -9, 1, 2, 3, 4},
 'eq': {-8, -7, -6, -5, 5, 6, 7, 8},
 'lt': {-4, -3, -2, -1, 9, 10, 11, 12}}

So now we have a viable approach to implementing `find_tree`:

   - We call `find_tree(puzzle, oddballs, weighings)`. At the top level, the oddballs and number of weighings come from the puzzle. At recursive levels, we will reduce the number of oddball possibilities according to the partition, and the number of remaining weighings by 1 each time.
   - At each step we will randomly select two groups of balls, `L` and `R`, to be weighed.
   - We will then see what partition `L` and `R` gives us, and whether the partition is good or bad.
   - If the partition is bad, try another random selection of `L` and `R`.
   - Use a **greedy** approach where we accept the first good partition.
      (If we don't find a good partition after 1,000 tries, we give up.)
   - Once we have a good partition, we recursively find a tree for each of the branches of the partition.

In [9]:
def find_tree(puzzle, oddballs, weighings) -> Tree or Oddball:
    "Find a strategy tree that covers all the oddballs in the given number of weighings."
    if len(oddballs) == 1:
        return oddballs.pop() # One oddball possibility left; we're done: leaf node
    elif len(oddballs) == 0:
        return 0              # No oddball
    elif weighings == 0:
        return None           # Can't find a solution in the allowable weighings
    else:
        L, R, part = find_good_partition(puzzle, oddballs, weighings - 1)
        if not part: 
            return None
        subtrees = {r: find_tree(puzzle, part[r], weighings - 1) for r in part}
        if None not in subtrees.values(): 
            return Tree(L, R, **subtrees)
    
def find_good_partition(puzzle, oddballs, weighings) -> (list, list, dict):
    "Return (L, R, partition) such that no partition entry has more than 3**weighings oddballs."
    for _ in range(1000): 
        L, R = random_LR(puzzle, oddballs)
        part = partition(L, R, oddballs)
        if all(len(entry) <= 3 ** weighings for entry in part.values()):
            return L, R, part
    return [], [], {} # Fail
    
def random_LR(puzzle, oddballs):
    "Random choice of balls for L and R side."
    # Pick a random number of balls, B, then randomly pick B balls for each side.
    B = random.choice(range(1, (len(puzzle.balls) - 1) // 3 + 2))
    random.shuffle(puzzle.balls) 
    return puzzle.balls[:B], puzzle.balls[-B:]

Here we see that `find_good_partition` does its job:

In [10]:
find_good_partition(p12, p12.oddballs, 2)

([9, 3, 2, 1],
 [10, 11, 4, 12],
 {'gt': {-12, -11, -10, -4, 1, 2, 3, 9},
  'eq': {-8, -7, -6, -5, 5, 6, 7, 8},
  'lt': {-9, -3, -2, -1, 4, 10, 11, 12}})

It uses `random`, so it won't get the same result every time:

In [11]:
find_good_partition(p12, p12.oddballs, 2)

([8, 12, 10, 4],
 [2, 6, 7, 11],
 {'gt': {-11, -7, -6, -2, 4, 8, 10, 12},
  'eq': {-9, -5, -3, -1, 1, 3, 5, 9},
  'lt': {-12, -10, -8, -4, 2, 6, 7, 11}})

# Solving Some Puzzles

Now we're ready to solve puzzles!

In [12]:
solve(p12)

Tree(L=[6, 2, 3, 5], R=[11, 1, 8, 4], gt=Tree(L=[3, 5, 11], R=[6, 4, 2], gt=Tree(L=[8, 11, 4, 5], R=[10, 9, 6, 7], gt=5, eq=3, lt=-4), eq=Tree(L=[6, 1, 9], R=[7, 12, 4], gt=0, eq=-8, lt=-1), lt=Tree(L=[1, 10, 5], R=[2, 11, 7], gt=-11, eq=6, lt=2)), eq=Tree(L=[12, 2, 3, 4], R=[10, 8, 7, 11], gt=Tree(L=[7, 4], R=[10, 5], gt=-10, eq=12, lt=-7), eq=Tree(L=[8, 1, 10, 11], R=[5, 12, 6, 9], gt=-9, eq=0, lt=9), lt=Tree(L=[12, 4, 8, 7], R=[11, 1, 3, 6], gt=7, eq=10, lt=-12)), lt=Tree(L=[2, 6, 8, 11], R=[10, 9, 5, 4], gt=Tree(L=[12, 11, 10], R=[8, 3, 4], gt=11, eq=-5, lt=8), eq=Tree(L=[8, 5, 1], R=[9, 12, 2], gt=1, eq=-3, lt=0), lt=Tree(L=[5, 10], R=[4, 2], gt=-2, eq=-6, lt=4)))

OK, that's hard to read&mdash;my bad. Let's look at a much easier puzzle: 3 balls, 1 weighing allowed, and the odd ball can only be lighter:

In [13]:
tree = solve(Puzzle(3, 1, {-1}))
tree

Tree(L=[2], R=[1], gt=-1, eq=-3, lt=-2)

This tree says you weigh one ball against another (leaving the third unweighed), and the three possible weighing results tell you which of the three balls  is lighter. 



# Prettier Output

Let's make the output easier to read. I'll use `[1·2·3·4 ⟘ 9·10·11·12] ➔` to mean *"The result of weighing balls 1,2,3,4 versus 9,10,11,12 is..."*. I'll indent for each interior node in the tree. Also, note that at the top node of a tree, there's no sense randomly shuffling the balls&mdash;the only choice that matters is how many balls, `B`, to put on each side. Putting `1·2·3·4` on the left is no different than `3·7·9·12`, because each ball is undifferentiated at the start. I'll alter `random_LR` to accomodate that.

In [14]:
def do(puzzle):
    "Print the solution to the puzzle as indented text."
    print(indented(solve(puzzle)))
    
def indented(tree, i=0, prefix=''):
    "Pretty, indented string of a strategy tree."
    if isinstance(tree, Tree):
        subtrees = f"{indented(tree.gt, i+1, '>:')} {indented(tree.eq, i+1, '=:')} {indented(tree.lt, i+1, '<:')}"
        indent   = '' if i == 0 else ('\n' + " " * 5 * i)
        return f'{indent}{prefix}[{items(tree.L)} ⟘ {items(tree.R)}] ➔ {subtrees})'
    elif tree == 0 or tree == None:
        return f'{prefix}{tree}'
    else:
        return f'{prefix}{tree:+d}'
    
def items(collection): return '·'.join(map(str, sorted(collection)))

def random_LR(puzzle, oddballs):
    "Random choice of balls for L and R side."
    # Pick a random number of balls, B, then pick B balls for each side.
    B = random.choice(range(1, (len(puzzle.balls) - 1) // 3 + 2))
    if oddballs == puzzle.oddballs:
        puzzle.balls.sort()
    else:
        random.shuffle(puzzle.balls) 
    return puzzle.balls[:B], puzzle.balls[-B:]

In [15]:
# 3 balls, 1 weighing, only lighter balls possible
do(Puzzle(3, 1, {-1}))

[1 ⟘ 3] ➔ >:-3 =:-2 <:-1)


In [16]:
# 3 balls, 2 weighings, lighter or heavier balls possible
do(Puzzle(3, 2))

[1 ⟘ 3] ➔ 
     >:[3 ⟘ 2] ➔ >:0 =:+1 <:-3) 
     =:[2 ⟘ 3] ➔ >:+2 =:0 <:-2) 
     <:[3 ⟘ 2] ➔ >:+3 =:-1 <:0))


In [17]:
# The original puzzle with 12 balls
do(p12)

[1·2·3·4 ⟘ 9·10·11·12] ➔ 
     >:[2·4·5·10 ⟘ 1·3·6·8] ➔ 
          >:[5 ⟘ 4] ➔ >:0 =:+2 <:+4) 
          =:[1·11 ⟘ 7·12] ➔ >:-12 =:-9 <:-11) 
          <:[3·9 ⟘ 1·12] ➔ >:+3 =:-10 <:+1)) 
     =:[2·5·10·11 ⟘ 4·6·8·9] ➔ 
          >:[8·11 ⟘ 1·6] ➔ >:-6 =:+5 <:-8) 
          =:[7·8 ⟘ 1·6] ➔ >:+7 =:0 <:-7) 
          <:[7·11·12 ⟘ 5·6·9] ➔ >:-5 =:+8 <:+6)) 
     <:[2·10·11 ⟘ 1·7·9] ➔ 
          >:[5·7·8·10 ⟘ 3·4·11·12] ➔ >:+10 =:-1 <:+11) 
          =:[1·5·8·10 ⟘ 2·3·11·12] ➔ >:-3 =:-4 <:+12) 
          <:[4·5·7·12 ⟘ 2·3·9·10] ➔ >:-2 =:0 <:+9)))


# Other Puzzles



We can solve much larger puzzles. We can do 39 balls with 4 weighings:

In [18]:
do(Puzzle(39, 4))

[1·2·3·4·5·6·7·8·9·10·11·12·13 ⟘ 27·28·29·30·31·32·33·34·35·36·37·38·39] ➔ 
     >:[3·4·6·7·8·12·21·27·29·33·34·36·37 ⟘ 10·11·14·16·18·22·23·24·25·26·28·30·35] ➔ 
          >:[2·9·12·13·17·18·20·26·31·35·39 ⟘ 1·3·6·10·19·21·24·28·29·30·37] ➔ 
               >:[1·3·5·11·13·15·16·20·24·29·35·38 ⟘ 7·8·9·10·12·17·26·27·28·34·36·39] ➔ >:-28 =:-30 <:+12) 
               =:[3·8·14·20·22·24·28·36·38 ⟘ 4·9·12·18·19·25·29·30·31] ➔ >:+8 =:+7 <:+4) 
               <:[5·11·13·15·18·19·20·26·30·32·34·36 ⟘ 1·4·6·7·16·21·25·27·29·31·35·38] ➔ >:-35 =:+3 <:+6)) 
          =:[3·7·12·14·15·19·20·23·25·26·29·34 ⟘ 1·4·5·9·11·18·24·30·31·37·38·39] ➔ 
               >:[2·6·16·22·26·31·32·33 ⟘ 1·3·9·11·12·13·37·38] ➔ >:-38 =:-39 <:-31) 
               =:[2·6·10·16·19·23·26·28·31·32 ⟘ 5·7·11·12·22·24·30·33·35·37] ➔ >:+2 =:+13 <:-32) 
               <:[4·5·10·19·23·31·36 ⟘ 6·8·9·11·33·38·39] ➔ >:+5 =:+1 <:+9)) 
          <:[2·5·6·7·11·15·18·22·25·28·29·30 ⟘ 3·9·10·13·17·20·23·26·31·33·35·36] ➔ 
               >:

However, we can't do 40 balls:

In [19]:
do(Puzzle(40, 4))

None


We can do 12 balls in 3 weighings even if it is possible that none of the balls is odd:

In [20]:
do(Puzzle(12, 3, {-1, 0, +1}))

[1·2·3·4 ⟘ 9·10·11·12] ➔ 
     >:[3·4·9·10 ⟘ 1·6·8·12] ➔ 
          >:[4·6·9·12 ⟘ 1·2·5·10] ➔ >:+4 =:+3 <:-12) 
          =:[1·5·11·12 ⟘ 4·6·9·10] ➔ >:0 =:+2 <:-11) 
          <:[1·9 ⟘ 4·11] ➔ >:+1 =:-10 <:-9)) 
     =:[1·7·9·11 ⟘ 5·8·10·12] ➔ 
          >:[4·7·8 ⟘ 2·11·12] ➔ >:+7 =:-5 <:-8) 
          =:[4·5·7·8 ⟘ 1·2·6·9] ➔ >:-6 =:0 <:+6) 
          <:[8·12 ⟘ 3·5] ➔ >:+8 =:-7 <:+5)) 
     <:[3·6·8·10 ⟘ 2·4·7·12] ➔ 
          >:[5·7·9 ⟘ 4·10·11] ➔ >:-4 =:-2 <:+10) 
          =:[5·11 ⟘ 3·9] ➔ >:+11 =:-1 <:+9) 
          <:[3·4·9 ⟘ 7·8·11] ➔ >:0 =:+12 <:-3)))


We can do **26** balls in 3 weighings if we know that no ball can be heavier (and one or none is lighter):

In [21]:
do(Puzzle(26, 3, {-1, 0}))

[1·2·3·4·5·6·7·8·9 ⟘ 18·19·20·21·22·23·24·25·26] ➔ 
     >:[3·7·12·14·15·18·21·25 ⟘ 1·6·8·10·17·20·24·26] ➔ 
          >:[3·6·9·10·13·14·17·26 ⟘ 5·7·8·15·21·23·24·25] ➔ >:-24 =:-20 <:-26) 
          =:[5·6·7·9·12·13·14·15·23 ⟘ 1·2·3·4·8·16·18·19·25] ➔ >:-19 =:-22 <:-23) 
          <:[1·2·7·9·10·12·14·19·25 ⟘ 3·4·5·8·13·15·16·21·22] ➔ >:-21 =:-18 <:-25)) 
     =:[10·11·16·19·20·21·22·23·24 ⟘ 1·2·3·4·7·13·14·17·26] ➔ 
          >:[2·5·7·14·15·16 ⟘ 4·9·11·17·19·21] ➔ >:-17 =:-13 <:-14) 
          =:[5·8·10·13·14·15·23·26 ⟘ 4·7·9·12·18·19·24·25] ➔ >:-12 =:0 <:-15) 
          <:[6·10·13·15·19·21·22·23·25 ⟘ 3·4·7·8·9·11·12·17·24] ➔ >:-11 =:-16 <:-10)) 
     <:[3·4·8·11·12·16·20 ⟘ 1·2·5·10·13·21·26] ➔ 
          >:[1·9·16·18·20·22 ⟘ 4·5·8·13·14·19] ➔ >:-5 =:-2 <:-1) 
          =:[1·4·7·13·16·17 ⟘ 5·9·10·12·21·25] ➔ >:-9 =:-6 <:-7) 
          <:[4·6·12·14·18 ⟘ 8·11·13·19·20] ➔ >:-8 =:-3 <:-4)))


Three weighings can theoretically handle at most 27 possibilities, and we used all 27 in the problem above. 

Can we solve the 13-ball, heavier-or-lighter problem, which has 26 possibilities? 

In [22]:
do(Puzzle(13))

None


**No**, and the reason is that there is no first weighing that partitions the 26 possibilities into 9/9/8; the best we can do is partition into 8/10/8 or 10/6/10:

In [23]:
partition([1, 2, 3, 4], [10, 11, 12, 13], Puzzle(13).oddballs)

{'gt': {-13, -12, -11, -10, 1, 2, 3, 4},
 'eq': {-9, -8, -7, -6, -5, 5, 6, 7, 8, 9},
 'lt': {-4, -3, -2, -1, 10, 11, 12, 13}}

In [24]:
partition([1, 2, 3, 4, 5], [9, 10, 11, 12, 13], Puzzle(13).oddballs)

{'gt': {-13, -12, -11, -10, -9, 1, 2, 3, 4, 5},
 'eq': {-8, -7, -6, 6, 7, 8},
 'lt': {-5, -4, -3, -2, -1, 9, 10, 11, 12, 13}}

Here's a puzzle with 25 balls, and the possibilities are that either one of the odd-numbered balls is heavier, or one of the even-numbered balls is lighter, or all the balls weigh the same.

In [25]:
p = Puzzle(25, 3)
p.oddballs = {(+b if b % 2 else -b) for b in p.balls} | {0}

do(p)

[1·2·3·4·5·6·7·8·9 ⟘ 17·18·19·20·21·22·23·24·25] ➔ 
     >:[1·5·8·9·11·13·18·22 ⟘ 2·6·7·14·16·17·19·25] ➔ 
          >:[2·4·9·10·11·13·16·21·22 ⟘ 3·5·8·12·18·19·20·23·24] ➔ >:+9 =:+1 <:+5) 
          =:[3·6·8·11·14·21·24 ⟘ 5·7·10·16·18·19·25] ➔ >:+3 =:-20 <:-24) 
          <:[1·2·5·10·12·14·19·21·23 ⟘ 6·7·9·11·13·18·20·24·25] ➔ >:-18 =:-22 <:+7)) 
     =:[7·9·10·15·17·20·25 ⟘ 2·3·11·12·16·18·23] ➔ 
          >:[4·6·11·12·13·19·22·23 ⟘ 9·16·17·18·20·21·24·25] ➔ >:-16 =:+15 <:-12) 
          =:[1·2·6·9·15·17·21·23·25 ⟘ 3·8·10·11·12·13·14·19·22] ➔ >:-14 =:0 <:+13) 
          <:[11·18·19 ⟘ 3·13·20] ➔ >:+11 =:-10 <:0)) 
     <:[2·9·12·13·17·20·23 ⟘ 4·5·15·18·21·22·25] ➔ 
          >:[1·2·10·14·16·17·21·25 ⟘ 5·7·15·18·19·22·23·24] ➔ >:+17 =:-4 <:+23) 
          =:[1·4·8·11·13·21 ⟘ 2·5·6·18·23·24] ➔ >:-6 =:+19 <:-8) 
          <:[6·9·14·17·21·23 ⟘ 5·12·15·18·20·25] ➔ >:+21 =:-2 <:+25)))


# What's Next?

- What other puzzles can you solve?
- Can you make a table of solvable and unsolvable puzzles?
- What happens when it is a possibility that *two or more* balls are odd?
- Can you prove which puzzles are unsolvable? Can you modify the `find_tree` function so that it only fails when the puzzle really is unsolvable, and not when it happens to have bad luck with `random`?
- What else can you discover?
